#### Imports

[Small description of all the imports we do]

In [ ]:
# Useful starting lines
import numpy as np
import seaborn as sns
from util.dataloader import *
from util.finance import *
from util.plots import *
from util.finance import stock, compare
from util.quotebankexploration import *
from util.wikipedia import *
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
import numpy as np

from util.apple_stores import *

from task1 import *
from task2 import *
from task3 import *
from task4 import *

pio.renderers.default = "notebook_connected"

%load_ext autoreload
%autoreload 2

# <a class="anchor" id="TOC"></a> Table of Contents

***

* 0. [Introduction](#intro)
* 1. [Loading the datasets](#sect_1)
* 2. [First look into the Apple stock market and related quotes](#sect_2)
* 3. [Impact of the number of pageviews on the speakers' Wikipedia page](#sect_3)
    * 3.1 [Wiki labels and wiki ID](#sect_3_1)
    * 3.2 [Number of wiki page views](#sect_3_2)
    * 3.3 [Exact label for each quotes](#sect_3_3)
    * 3.4 [Scoring quotes](#sect_3_4)
    * 3.5 [Get back sentiment analysis](#sect_3_5)
    * 3.6 [Final plot](#sect_3_6)

# 0. Introduction <a class="anchor" id="intro"></a>
[Back to table of contents](#TOC)

***

This notebook is intended to show and demonstrate the thought process that went into this project, on identifying patterns and correlations in stock markets and media quotes. In particular we have focused on the Apple Stock market, as it is one of the most valuable company on earth and it is widely covered in the media. 

# 1. Loading the datasets <a class="anchor" id="sect_1"></a>
[Back to table of contents](#TOC)

***

In this project we have studied three datasets to provide more insights in the stock price evolution, the coverage of Apple in the media, and its relationship with the various speakers. This first session is dedicated to loading the various datasets and preprocessing the valuable information.

### Apple stock : yFinance API
The yFinance API is provided by Yahoo Finance and provides an easy access to various financial metrics for most stocks in the market. The _ticker_ of the Apple stock is denoted _AAPL_, and we will first focus on a date range from 2010 up to 2020. We also provide an additional indicator of the daily volatility with the ``Liquidity`` field, which is the daily volume multiplied by the average daily stock price. This indicator in dollars is appropriate to have a quick overview on the quantity of Apple stock traded in a day, as a day of high liquidity is also said to be a day of high volatility. 

In [ ]:
# Using yFinance API we load various metrics of the Apple stock ranging from 2008 to 2020
stock = load_stock("AAPL", 2015, 2020)

# We set a day of high volatility as a day among the highest 2% of liquidity in that year.
stock = high_volatility(stock, quantile = 0.98)

display(stock)

## Quotebank Dataset
The Quotebank data is a large text corpus of more than 178 millions quotations scrapped over 337 websites. As we are focusing on Apple related quotations, we have applied a various amount of filtering to reduce the number of quotes to 310'816. Some of the various techniques employed: 
1. White list of words that should be contained in quotes : _Apple_, _iPhone_, _Macbook_ etc.
2. Black list of words that should not be contained : _Mac n Cheese, _apple_, _Big Apple_ etc.
3. White list of speakers, as we also included speakers related to Apple regardless of the aforementioned white words : _Steve Jobs_, _Tim Cook_, _Steve Wozniak_ etc.

This final dataset of Apple related quotes have been saved in a `pkl` file to improve the ease of manipulation, and can be accessed with the `get_filtered_quotes` function.

In [ ]:
quotes = get_filtered_quotes()

display(quotes)

## Wikipedia

wallah je sais pas

In [ ]:
print('caca yolo prout')

# 2. First look into the Apple stock market and related quotes <a class="anchor" id="sect_2"></a>
[Back to table of contents](#TOC)

***

### 2.1 Observations on the volatility, liquidity traded and media discussion
 
One of the objectives of this project is to find patterns and events in both the stock market and media discussion related to Apple. To proceed on this, we need metrics and qualitative visualization of the aforementioned data. To find days of interest in the stock market, we introduce the _Liquidity_, which is the mean price day multiplied by the daily volume of exchange. This indicator provides an intuition of the amount of $AAPL stock exchanged in day. A day of high liquidity will be synonymous of a day of high volatility, which may indicate a particular event related to Apple.

In the following plot we highlight the yearly top 2% of the liquidity days. In other words, we will observe each year the days with the highest liquidity. We observe a pattern that will be further studied, as the iPhone september events, or the quarterly reports. 

In [ ]:
weekly_liquidity(stock, quantile=0.98)

Naturally we proceed on this comparison of high exchange days by looking at the daily price of the $AAPL stock. We highlight the same days of high volatility from the previous figures but next to the stock price instead of the liquidity. The intuition is that a day of high volatility may have repercussions on the stock price, either as a price fall or a rebound. 

In [ ]:
daily_stock_price(stock)

Finally we will take a look at the filtered Quotebank dataset, which only describes Apple related quote. In the same idea as previous plots, we can highlight the yearly top 2% of days with the most quotes. Again we can observe some patterns, that will be further dissected in the next few sections. For example we observe a yearly spike in September when the new iPhone is released, or in June with the yearly developer conference. Most importantly the highest spike the 6th October 2011 is related to Steve Jobs death, which was widely covered in the media.

In [ ]:
daily_quotes(quotes, quantile = 0.98)

### 2.2 Finding patterns and studying correlated features

From the above observations, we had the intuition that each quarter report release by Apple was synonymous with a day of high volatility. A quarterly report is a summary or collection of unaudited financial statements, such as  gross revenue, net profit, operational expenses, and cash flow. As we have 252 trading days in a year and 4 quarter reports per year, we expect a periodicity of high liquidity days of around $252/4 = 64$ days. This in fact validated by the next cell, which performs a seasonal analysis over a wide range of days period, and keep the days with the lowest null probability (i.e p-value). Later on we will see if these pattern is also observable on the Apple related quotes.

In [ ]:
seasonal_analysis(stock[stock.Date.dt.year.isin([2018,2019])], column="Liquidity")

In [ ]:
stock_price_with_quotes(stock,quotes)

What we finally observe is that there is a qualitative correlation between the number of daily quotes related to Apple and the daily liquidity. The two are positively correlated, which indicates that an increase in liquidity will be likely associated with an increase in discussion related to Apple and the other way around. 

In [ ]:
corr, p_value = pearson_stock_quotes(stock[stock.Date.dt.year.isin(range(2015,2019))],quotes)

print(f"The correlation between the stock liquidity and the daily number of quotes is {corr:.3f} with {p_value} p_value")

# 3. Impact of the number of pageviews on the speakers' Wikipedia page <a class="anchor" id="sect_3"></a>
[Back to table of contents](#TOC)

***

Now we have looked the positiveness of the different quotes of our data set, we want to add more depth about on the impact of the quotes on the stock market. To do so, we pose to ourselves the following question: What is the impact of a quote on others ? The response is relatively simple, it depends on how well known the speaker was at the time he or she was quoted in the media. And a quite simple way to have that indicators is the number of pageviews on then speakers' Wikipedia page (if there is one !).

#### 3.1 Wiki labels and wiki ID <a class="anchor" id="sect_3_1"></a>
[Back to table of contents](#TOC)

But before accessing to this number of pageviews, we need the exact label of the speakers' Wikipedia page. And for that, we will load the following data set that particularly contains the precise label's page for its ocrresponding wiki ID that we have for each quotes in our quotes' data set.

In [ ]:
# Get the wiki labels with its corresponding wiki ID
wiki_labels = get_wiki_labels()[['id', 'label']]
display(wiki_labels.head(5))

#### 3.2 Number of wiki page views <a class="anchor" id="sect_3_2"></a>
[Back to table of contents](#TOC)

After that, we need a way to get the number of pageviews of some speaker for a specific year. For that we use the package `pageviewapi` that gives us all the wiki page views from 2015. We would have liked to have all the pageviews since 2008, but it was too complicated and one person cerate a way to get these data, but the website was not able to work anymore. Then, we had focus our study between 2015 and 2020, and we had design a fucntion that returns the number of pageviews of a specific wiki page and year. This is what follows.

In [ ]:
# Get the page views for the Steve Jobs wikipedia page in 2015
speaker = 'Steve Jobs'
year = 2015
print(get_page_views_per_year(speaker, year))

#### 3.3 Exact label for each quotes <a class="anchor" id="sect_3_3"></a>
[Back to table of contents](#TOC)

The idea now is to get the exact label for each speaker of every single quotes in our quotes data set. This can be done directly by merging the ID in `wiki_labels` with the QID of the data frame `quotes`, but here it's not that simple. The different quotes of our data set given by quotebank has more than one QID because the name of the speaker can be confusing with another speaker, so it gives the two QID in the list for that quote. To deal with this issue, we decided to use look at all the different QID for each quotes, and keep the label that corresponds to the speaker having the maximum number of total pageviews. The following cell show how the process is done over a sample the dataframe `quotes`. We do not do the whole filtering here because the run is about 22h, so wa done it once on multiple clusters and save it in a `.pkl` file.

In [ ]:
# Get the right label of the speakers of each quotes.
# It adds a new column `label` in quotes dataframe containig the wiki label of the speaker.
quotes_label_sample = find_labels(quotes.head(5), wiki_labels)
display(quotes_label_sample)

#### 3.4 Scoring quotes <a class="anchor" id="sect_3_4"></a>
[Back to table of contents](#TOC)

Now we have the exact label of the spaker of each quotes, we want to get the number of the speaker's wiki page views at the year where the quote was published. So the idea is simple: we just take the label and the year of each quotes and use the function `get_pageviews_per_year` to add a new column `score` in our data frame.

**Remark :** As in the previous subsection, here is an example ofhow the process works on a small smaple of the data set because is too long (around 8h for this one). Since the large runnning time, we decided to split the steps such taht we can run it on multiple computer. The idea was then, to get pageviews for each speaker for every year between 2015 and 2020. Here is a sample of how the code works.

In [ ]:
# Apply the process on a small sample
speakers_pageviews_sample = get_speakers_pageviews_per_year(speakers_labels_sample)
display(speakers_pageviews_sample)

# Load the whole data set from a pickle file
speakers_pageviews = get_speakers_pageviews()

Now, we directly join the year of the quotes with the corresponding number of pageviews of the speakers. After that we normalize the value to be sure to have a score that is relevant.

In [ ]:
# Add score column fro our data frame
quotes_score_sample = get_score_quotes(quotes, speakers_pageviews)
display(quotes_score_sample.head(5))

# Get the whole data set from a .pkl file

#### 3.5 Get back sentiment analysis <a class="anchor" id="sect_3_5"></a>
[Back to table of contents](#TOC)

In [ ]:
# Use the code of Camille

#### 3.6 Final plot <a class="anchor" id="sect_3_6"></a>
[Back to table of contents](#TOC)

In [ ]:
# Plot

----
# 5. Building a model for stock market prediction
[Back to table of contents](#TOC)

Using the quotes related to Apple, speakers data and past stock performance, we can perform a first attempt at predicting the daily stock price and the liquidity. For this section we will use the Facebook's Prophet library, which provides powerful and easy to use forecasting tools. At its core, the model is a modular linear regression model, that can take into account past performance and additional factors. 

### 5.1 Model Fitting
[Back to table of contents](#TOC)


In this section we will fit and test in the range from 2015 to 2019. We have seen in a 2020 a stock split of $AAPL, which can throw off the model prediction. We will train our model between 2015 and 2018, and test its predictive capability in 2019. The obtained model will be further validated through forecasting cross validation. 

In [ ]:
# Load the sentiment and quotes features obtained in previous sections
quotes_sentiment = pd.read_pickle("data/quotes_score.pkl")

# Build the train dataframe of stock, sentiment and quotes features
prediction_frame = build_prediction_frame(stock[stock.Date.dt.year.isin(range(2015,2018))],quotes_sentiment)

# Fit the model on the train dataframe
m = fit_prophet(Prophet(changepoint_prior_scale=0.05, seasonality_prior_scale=0.1), prediction_frame, features=['positive','negative','total'], response='Open')

In [ ]:
# Predict the next 300 days of trading
pred = predict_future(m,prediction_frame,feature_frame=quotes_sentiment)

# Plot the predicted future 300 days compared to the true stock price
plot_prediction(stock, quotes_sentiment, pred)

### 5.2 Evaluate model performance
[Back to table of contents](#TOC)

Using cross validation, we can observe the performance of the model using a variety of error metrics. We will evaluate the performance of the model on a 60 days horizon, starting with a train set of 150 days and successively adding the next 30 days at each fold.

In [ ]:
# Estimate the 60 days future prediction using forecasting cross validation
df_cv = cross_validation(m, initial='150 days', period='30 days', horizon = '60 days',parallel="processes")

# Evaluate multiple metrics on the 60 days future prediction
df_p = performance_metrics(df_cv)

print("Mean absolute percentage error in a first week horizon", df_p["mape"].values[0])

In [ ]:
# Set the parameters to be evaluated
param_grid = {  
  'changepoint_prior_scale': np.linspace(0.2,2,5) ,
  'seasonality_prior_scale': np.logspace(-2,1,5),
}

# Performs grid search cross validation to estimate the mean MAPE for each pair of parameters
tuning_results = prophet_cross_validation(param_grid, stock, quotes_sentiment, features = ['positive','negative','total'], response = 'Open', metric = 'mape')

# Output the best pair of parameters in term of mean validation MAPE
tuning_results[tuning_results.mape == tuning_results.mape.min()]